In [0]:
# Stack location : ls
dbutils.fs.ls("dbfs:/FileStore/code-templates/")

Out[1]: [FileInfo(path='dbfs:/FileStore/code-templates/.git/', name='.git/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/README.md', name='README.md', size=9902),
 FileInfo(path='dbfs:/FileStore/code-templates/TigerML/', name='TigerML/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/ct_stack_setup/', name='ct_stack_setup/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/data/', name='data/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/delta_data/', name='delta_data/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/deploy/', name='deploy/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/logs/', name='logs/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/mlflow_artifacts/', name='mlflow_artifacts/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/mlruns/', name='mlruns/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/notebooks_config/', name='notebooks_config/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/output_reports/', name='output_reports/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/production/', name='production/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/production_config/', name='production_config/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/scripts/', name='scripts/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/setup.cfg', name='setup.cfg', size=587),
 FileInfo(path='dbfs:/FileStore/code-templates/setup.py', name='setup.py', size=799),
 FileInfo(path='dbfs:/FileStore/code-templates/src/', name='src/', size=0),
 FileInfo(path='dbfs:/FileStore/code-templates/tasks.py', name='tasks.py', size=24063)]

### Setting up Python Environment
##### INIT script
* It takes ~12 mins to start the cluster using this init script
* Please make sure init file should be written using dbutils.fs.put command only otherwise it can cause 'fi' syntax issue

In [0]:
dbutils.fs.put("dbfs:/FileStore/Git/init.sh","""
#!/bin/bash
echo $DB_IS_DRIVER
if [[ $DB_IS_DRIVER = "TRUE" ]]; then
  # Installing packages from conda
  /databricks/python/bin/python -V
  . /databricks/conda/etc/profile.d/conda.sh
  conda activate /databricks/python
  conda install --yes --file /dbfs/FileStore/Git/code-templates/deploy/conda_envs/linux-cpu-64-dev.lock
  # Installing packages from PIP
  pip install -r /dbfs/FileStore/Git/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt 
  # Installing Code-Templates
  cd /dbfs/FileStore/Git/code-templates
  pip install -e .
else
  /databricks/python/bin/pip install invoke==1.3.0
fi
""", True)

Wrote 611 bytes.
Out[23]: True

In [0]:
%sh
cat /dbfs/FileStore/Git/init.sh

#!/bin/bash
echo $DB_IS_DRIVER
if [[ $DB_IS_DRIVER = "TRUE" ]]; then
 # Installing Invoke package
 /databricks/python/bin/pip install invoke==1.3.0
 /databricks/python/bin/pip install luminol==0.4
 /databricks/python/bin/pip install ruptures==1.1.3
 # Installing packages from conda
 /databricks/python/bin/python -V
 . /databricks/conda/etc/profile.d/conda.sh
 conda activate /databricks/python
 conda install --yes --file /dbfs/FileStore/Git/code-templates/deploy/conda_envs/linux-cpu-64-dev.lock
 # Installing packages from PIP
 pip install -r /dbfs/FileStore/Git/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt 
 # Installing Code-Templates
 cd /dbfs/FileStore/Git/code-templates
 pip install -e .
 # Installing TigerML
 cd /dbfs/FileStore/Git/TigerML/python/
 pip install -e .
else
 /databricks/python/bin/pip install invoke==1.3.0
fi

##### Checking INIT Logs

Initiate logging in cluster advance setting to view this

In [0]:
%sh
# INIT file log
ls /dbfs/cluster-logs/0427-081739-mushy682/init_scripts/0427-081739-mushy682_10_139_64_4

20210427_102637_00_init.sh.stderr.log
20210427_102637_00_init.sh.stdout.log
20210427_103827_00_init.sh.stderr.log
20210427_103827_00_init.sh.stdout.log
20210427_104122_00_init.sh.stderr.log
20210427_104122_00_init.sh.stdout.log
20210427_104519_00_init.sh.stderr.log
20210427_104519_00_init.sh.stdout.log
20210427_110027_00_init.sh.stderr.log
20210427_110027_00_init.sh.stdout.log
20210427_110514_00_init.sh.stderr.log
20210427_110514_00_init.sh.stdout.log
20210427_110551_00_init.sh.stderr.log
20210427_110551_00_init.sh.stdout.log
20210427_163056_00_init.sh.stderr.log
20210427_163056_00_init.sh.stdout.log

In [0]:
%sh
cat /dbfs/cluster-logs/0427-081739-mushy682/init_scripts/0427-081739-mushy682_10_139_64_4/20210427_163056_00_init.sh.stderr.log

In [0]:
%sh
tail -f /dbfs/cluster-logs/0427-081739-mushy682/init_scripts/0427-081739-mushy682_10_139_64_4/20210427_163056_00_init.sh.stdout.log

shap-0.35.0 | | 0% 
shap-0.35.0 | ########## | 100% 

statsmodels-0.11.1 | | 0% 
statsmodels-0.11.1 | ########## | 100% 

tpot-0.11.5 | | 0% 
tpot-0.11.5 | ########## | 100% 

yellowbrick-1.1 | | 0% 
yellowbrick-1.1 | ########## | 100% 

category_encoders-2. | | 0% 
category_encoders-2. | ########## | 100% 

pyjanitor-0.20.9 | | 0% 
pyjanitor-0.20.9 | ########## | 100% 

pymc3-3.9.3 | | 0% 
pymc3-3.9.3 | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

## Specific commands of INIT

###### Command to install PIP packages

In [0]:
%sh
# Testing invoke commands
pip install -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt 

Collecting git+https://github.com/jupyter/jupyter-sphinx.git@v0.3.1 (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 31))
 Cloning https://github.com/jupyter/jupyter-sphinx.git (to revision v0.3.1) to /tmp/pip-req-build-kqr18817
Requirement already satisfied (use --upgrade to upgrade): jupyter-sphinx==0.3.1 from git+https://github.com/jupyter/jupyter-sphinx.git@v0.3.1 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 31))
Requirement already satisfied: great-expectations==0.13.4 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 1)) (0.13.4)
Requirement already satisfied: xverse==1.0.5 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 2)) (1.0.5)
Requirement already satisfied: cloudpickle==1.3.0 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 3)) (1.3.0)
Processing /root/.cache/pip/wheels/3a/34/2d/d78c1a3c467d348ed40de462777bb69f61dd3176ee940409fc/gast-0.3.2-py3-none-any.whl
Requirement already satisfied: absl-py==0.10.0 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 5)) (0.10.0)
Requirement already satisfied: astunparse==1.6.3 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 6)) (1.6.3)
Requirement already satisfied: google-pasta==0.2.0 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 7)) (0.2.0)
Requirement already satisfied: grpcio==1.32.0 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 8)) (1.32.0)
Requirement already satisfied: keras-preprocessing==1.1.2 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 9)) (1.1.2)
Requirement already satisfied: opt-einsum==3.3.0 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 10)) (3.3.0)
Requirement already satisfied: protobuf==3.13.0 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 11)) (3.13.0)
Requirement already satisfied: pyasn1==0.4.8 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 12)) (0.4.8)
Requirement already satisfied: rsa==4.6 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 13)) (4.6)
Requirement already satisfied: pyasn1-modules==0.2.8 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 14)) (0.2.8)
Requirement already satisfied: google-auth==1.22.1 in /databricks/conda/envs/databricks-ml/lib/python3.8/site-packages (from -r /dbfs/FileStore/code-templates/deploy/conda_envs/requirements-linux-cpu-64-dev.txt (line 15)) (1.22.1)
Requirement already satisfied: google-auth-oauthlib==0.4.1 in /databricks/conda/envs/databricks

###### Command to install Conda packages

In [0]:
%sh
/databricks/python/bin/python -V
. /databricks/conda/etc/profile.d/conda.sh
conda activate /databricks/python
conda install --yes --file /dbfs/FileStore/code-templates/deploy/conda_envs/linux-cpu-64-dev.lock

Python 3.7.7
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

###### Command to install code-template(ta_lib) packages

In [0]:
%sh
cd /dbfs/FileStore/code-templates
pip install -e .

Obtaining file:///dbfs/FileStore/code-templates
Installing collected packages: ta-lib
 Attempting uninstall: ta-lib
 Found existing installation: ta-lib 1.0.0
 Uninstalling ta-lib-1.0.0:
 Successfully uninstalled ta-lib-1.0.0
 Running setup.py develop for ta-lib
Successfully installed ta-lib

###### Command to install TigerML packages

In [0]:
%sh
# ls /dbfs/FileStore/code-templates/TigerML
cd /dbfs/FileStore/code-templates/TigerML
pip install -e .

Obtaining file:///dbfs/FileStore/code-templates/TigerML
Installing collected packages: tigerml
 Running setup.py develop for tigerml
Successfully installed tigerml